In [1]:
import GCRCatalogs 


In [2]:
cat_str = "baseDC2_snapshot_z1.01_v0.1"
def get_catalog_cosmology(cat_str):
    """
    Helper function to return catalog cosmology object and redshift
    """
    cat = GCRCatalogs.load_catalog(cat_str)
    return cat.cosmology
COSMO = get_catalog_cosmology(cat_str)




In [2]:
# Define helper function for keeping track of progress
def progress_bar(cur_val, final_val):
    """ 
    Function to keep track of progress during computations by displaying
    a progress bar

    Parameters:
    cur_val (int/float): current iteration/value calculation is on
    final_val (int/float): final iteration/value that calculation will take
    """

    bar_length = 20
    percent = float(cur_val) / final_val
    arrow = '-' * int(round(percent * bar_length)-1) + '>'
    spaces = ' ' * (bar_length - len(arrow))

    sys.stdout.write("\rProgress: [{0}]"
                     " {1}%".format(arrow + spaces, int(round(percent * 100))))
    sys.stdout.flush()
    

In [6]:
import sys,os
import h5py
import numpy as np
import pyccl
genericio_path = r"/global/homes/s/spandey/genericio/python/"
gcrcatalogs_path = r"/global/homes/s/spandey/gcr-catalogs/"
sys.path.append(genericio_path)
sys.path.append(gcrcatalogs_path)
import genericio
H0 = 71.
bmin = 0.0
bmax = 500.0

def load_dark_matter_positions(file_str):
    """
    Loads and applies preliminary filters to dark matter position data in Mpc
    from a specified catalog and then applies a Gaussian kernel

    Parameters:
    file_str (string): location of file to load dark matter particles from
    N (int):
    
    Returns:
    pos_mat (N*N*N array): grid containing number of particles in box 
    """
    
    # Load data from catalog using genericio
    x_data = np.array(genericio.gio_read(file_str, "x")[0])
    ind_selx = np.where((x_data > bmin) &  (x_data < bmax))
    y_data = np.array(genericio.gio_read(file_str, "y")[0])
    ind_sely = np.where((y_data > bmin) &  (y_data < bmax))
    z_data = np.array(genericio.gio_read(file_str, "z")[0])
    ind_selz = np.where((z_data > bmin) &  (z_data < bmax))
    return np.vstack((x_data[ind_selx]*100/H0, y_data[ind_sely]*100/H0, z_data[ind_selz]*100/H0)).T


        

OSError: /global/homes/s/spandey/genericio/python/../frontend/libpygio.so: cannot open shared object file: No such file or directory

In [ ]:
dark_matter_file_str = ("/global/projecta/projectdirs/lsst/groups/CS/"
                        "cosmoDC2/Outer_snapshots/z1.01/m000.mpicosmo.247")

N_part = int(10765080312/216) # Number of total DM particles in the catalog (pre-determined)

with h5py.File('/global/cfs/cdirs/m1727/shivamp_lsst/data_set/dc_matter/dm_cat_' + str(bmin) + '_' + str(bmax) + '.h5' , 'w') as ff:    
    
    pos = ff.create_dataset("Position", dtype=("f8"), shape=(N_part, 3))
    cur_index = 0

    # Iterate through all 256 DM catalog files 
    for i in range(256):

        # Load positions
        positional_data =  load_dark_matter_positions(dark_matter_file_str+"#"+str(i))
        
        # Save positions
        n_pos = positional_data.shape[0]
        if cur_index+n_pos < N_part:
            pos[cur_index:cur_index+n_pos] = positional_data
            cur_index += n_pos
        else:
            break
        # Update progress
        progress_bar(i, 256)

